In [35]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhi0990","key":"bb67086b8df494189d4fbcf544295ce6"}'}

In [36]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [37]:
!ls ~/.kaggle

kaggle.json


In [38]:
!kaggle datasets download -d bittlingmayer/amazonreviews

 96% 472M/493M [00:01<00:00, 279MB/s]
100% 493M/493M [00:01<00:00, 272MB/s]


In [39]:
!unzip amazonreviews.zip

Archive:  amazonreviews.zip
  inflating: test.ft.txt.bz2         
  inflating: train.ft.txt.bz2        


In [40]:
!rm kaggle.json
!rm amazonreviews.zip
!ls

sample_data  test.ft.txt.bz2  train.ft.txt.bz2


In [41]:
import bz2
import pandas as pd

# Path to the bz2 file on Kaggle
bz2_filepath1 = '/content/train.ft.txt.bz2'
bz2_filepath2 = '/content/test.ft.txt.bz2'
labels1 = []
labels2 = []
reviews1 = []
reviews2 = []
with bz2.open(bz2_filepath1, 'rt', encoding='utf-8') as f:
    for line in f:
        label, review = line.split(' ', 1)
        labels1.append(label)
        reviews1.append(review)

with bz2.open(bz2_filepath2, 'rt', encoding='utf-8') as f:
    for line in f:
        label, review = line.split(' ', 1)
        labels2.append(label)
        reviews2.append(review)

train = pd.DataFrame({
    "class": labels1,
    "reviews": reviews1
})
test = pd.DataFrame({
    "class": labels2,
    "reviews": reviews2
})
train['class'] = train['class'].replace({'__label__1': 0, '__label__2': 1})
test['class'] = test['class'].replace({'__label__1': 0, '__label__2': 1})


In [42]:
!rm test.ft.txt.bz2
!rm train.ft.txt.bz2

In [43]:
train.head()

,class,reviews
0,1,Stuning even for the non-gamer: This sound tra...
1,1,The best soundtrack ever to anything.: I'm rea...
2,1,Amazing!: This soundtrack is my favorite music...
3,1,Excellent Soundtrack: I truly like this soundt...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."


In [44]:
test.head()

,class,reviews
0,1,Great CD: My lovely Pat has one of the GREAT v...
1,1,One of the best game music soundtracks - for a...
2,0,Batteries died within a year ...: I bought thi...
3,1,"works fine, but Maha Energy is better: Check o..."
4,1,Great for the non-audiophile: Reviewed quite a...


In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600000 entries, 0 to 3599999
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   class    int64 
 1   reviews  object
dtypes: int64(1), object(1)
memory usage: 54.9+ MB


In [46]:
train.describe()

,class
count,3600000.0
mean,0.5
std,0.5
min,0.0
25%,0.0
50%,0.5
75%,1.0
max,1.0


In [47]:
train.isnull().sum()

class      0
reviews    0
dtype: int64

In [54]:
train["class"].unique()
train["class"].value_counts()

1    1800000
0    1800000
Name: class, dtype: int64

In [56]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=f45669d43def885b3979166bcc577c7b5a924c2eb9d3e010989f306c416ae425
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [62]:
# training purpose
# train = train.head(10)
# test = test.head(10)

In [63]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')

english_stopwords = set(stopwords.words('english'))
spanish_stopwords = set(stopwords.words('spanish'))

stemmer_english = SnowballStemmer("english")
stemmer_spanish = SnowballStemmer("spanish")

def preprocess_multilingual(text):
    words = word_tokenize(text.lower())

    processed_words = []
    for word in words:
        if word.isalpha():
            # Language detection (simplified for demonstration)
            lang = detect(word)

            # Remove stopwords and apply stemming based on detected language
            if lang == 'en' and word not in english_stopwords:
                processed_words.append(stemmer_english.stem(word))
            elif lang == 'es' and word not in spanish_stopwords:
                processed_words.append(stemmer_spanish.stem(word))

    return ' '.join(processed_words)
train['reviews'] = train['reviews'].apply(preprocess_multilingual)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
test['reviews'] = test['reviews'].apply(preprocess_multilingual)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
train.head()

,class,reviews
0,1,sound track would peopl hate play play fresher...
1,1,soundtrack anyth read lot soundtrack write yea...
2,1,soundtrack play person throughout year higher ...
3,1,soundtrack soundtrack play relax death ancient...
4,1,hear play know without can heart wrote


In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limiting to 5000 most frequent words for efficiency

# Fit and transform the preprocessed reviews
X1 = vectorizer.fit_transform(train['reviews'])
y1 = train['class']
X2 = vectorizer.transform(test['reviews'])
y2 = test['class']



In [73]:
X_train, X_test, y_train, y_test = X1,X2,y1,y2
print(X2)

  (0, 70)	0.6476888299953735
  (0, 53)	0.761904967498719
  (1, 64)	0.27402365226895076
  (1, 51)	0.8525781666664829
  (1, 44)	0.19141584657999192
  (1, 28)	0.23973831517489266
  (1, 7)	0.32234612086385156
  (2, 6)	1.0
  (4, 42)	0.7071067811865475
  (4, 14)	0.7071067811865475
  (5, 70)	0.6476888299953735
  (5, 2)	0.761904967498719
  (6, 70)	0.4472135954999579
  (6, 65)	0.8944271909999159
  (7, 9)	0.8619580279084831
  (7, 3)	0.5069796427117355
  (8, 50)	1.0
  (9, 67)	0.8360479787483035
  (9, 34)	0.41802398937415175
  (9, 9)	0.35535858163071754


In [74]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [75]:
from sklearn.metrics import accuracy_score, classification_report
print("Logistic Regression")
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.5

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.50      1.00      0.67         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
print("Random Forest")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy_rf)
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))


Accuracy: 0.5

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.50      1.00      0.67         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
